In [1]:
#!/usr/bin/env python3
import scipy.io.wavfile
from python_speech_features import mfcc, delta
import numpy as np
import math

import tensorflow as tf
from tensorflow.keras import layers

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
np.random.seed(1234)

#files = [ 'vorst_14_machiavelli_8khz.wav'
# ]
files = [ 'cousinhenry_01_trollope_8khz.wav',
'siegeofcorinth_2_byron_8khz.wav',
'upperroom_16_ryle_8khz.wav',
'vorst_14_machiavelli_8khz.wav',
]

#more comments about this calculation? +1 - bias?
height_of_one_training_example = 49 * 13 * 2 + 1

label = 0
all_examples = []
for one_file in files:
  label += 1
  rate, data = scipy.io.wavfile.read(one_file)
  total_length_of_wave = data.shape[0]
  print ("just read file number %d which contains %d audio samples and is named %s Now analying it:" % (label, total_length_of_wave, one_file))
  assert rate == 8000, "rate was %d" % rate

  half_second_length = 4000
  start_index_of_half_second = 0
  num_training_example_in_this_file = 0
  while total_length_of_wave - start_index_of_half_second >= half_second_length:
    num_training_example_in_this_file += 1
    if num_training_example_in_this_file % 500 == 0:
      print ("\t analyzing training sample number %d" % num_training_example_in_this_file)

    this_training_example_raw = data[start_index_of_half_second:start_index_of_half_second + half_second_length]
    start_index_of_half_second += half_second_length
    assert len(this_training_example_raw) == 4000, len(this_training_example_raw)
    mfccs = mfcc(this_training_example_raw, 8000)
    assert mfccs.shape == (49, 13), mfccs.shape

    #Alfredo used 2 here, and changing it doesn't change the output size.
    first_derivative = delta(mfccs, 2)
    assert first_derivative.shape == (49, 13), first_derivative.shape
    all_examples.extend(mfccs.flatten().tolist())
    all_examples.extend(first_derivative.flatten().tolist())
    all_examples.append(label)
    assert len(all_examples) % height_of_one_training_example == 0, "num_training_example_in_this_file = %d" % num_training_example_in_this_file

all_examples_np = np.array(all_examples)
all_examples_np = all_examples_np.reshape((height_of_one_training_example, -1), order='F')

#print ("all_examples_np.shape = %s, so we have %d training samples" % (all_examples_np.shape, all_examples_np.shape[1]))
assert all_examples_np[-1, 0] == 1, "make sure the last row labels the first column as belonging to file number 1 %s" % all_examples_np[-1, 0]

shuffled_examples = all_examples_np.T
np.random.shuffle(shuffled_examples)
shuffled_examples = shuffled_examples.T

training_pct = 0.8

number_of_training_examples = int(math.ceil(all_examples_np.shape[1] * training_pct))

X_train = shuffled_examples[0:-1, 0:number_of_training_examples]
Y_train = shuffled_examples[-1:, 0:number_of_training_examples]
X_dev   = shuffled_examples[0:-1, number_of_training_examples:]
Y_dev   = shuffled_examples[-1:, number_of_training_examples:]

just read file number 1 which contains 7305509 audio samples and is named cousinhenry_01_trollope_8khz.wav Now analying it:
	 analyzing training sample number 500
	 analyzing training sample number 1000
	 analyzing training sample number 1500
just read file number 2 which contains 12400013 audio samples and is named siegeofcorinth_2_byron_8khz.wav Now analying it:
	 analyzing training sample number 500
	 analyzing training sample number 1000
	 analyzing training sample number 1500
	 analyzing training sample number 2000
	 analyzing training sample number 2500
	 analyzing training sample number 3000
just read file number 3 which contains 36554719 audio samples and is named upperroom_16_ryle_8khz.wav Now analying it:
	 analyzing training sample number 500
	 analyzing training sample number 1000
	 analyzing training sample number 1500
	 analyzing training sample number 2000
	 analyzing training sample number 2500
	 analyzing training sample number 3000
	 analyzing training sample number 3

In [15]:
# Xs are shape (number of input features, number of data points)
# Ys are shape (1, number of data points)
# The labels in Y are an integer corresponding to the speaker number.
# Before reshape
# (1274, 11853) (1, 11853) (1274, 2963) (1, 2963)

# In Keras, you want (number of data, attributes)
# Want: (see coursera M4 - Keras Tutorial)
# (11853, 1274) (11853, 1) (2963, 1274) (2963, 1)
# Reshape
X_train = X_train.T
Y_train = Y_train.T
X_dev = X_dev.T
Y_dev = Y_dev.T

#check
print(X_train.shape, Y_train.shape, X_dev.shape, Y_dev.shape)

#debugging - to be deleted
print(Y_train[100:120])

(11853, 1274) (11853, 1) (2963, 1274) (2963, 1)
[[3.]
 [3.]
 [3.]
 [1.]
 [3.]
 [2.]
 [3.]
 [3.]
 [2.]
 [3.]
 [3.]
 [3.]
 [2.]
 [3.]
 [3.]
 [3.]
 [2.]
 [3.]
 [3.]
 [3.]]


In [22]:
#Keras
model = tf.keras.Sequential()

#hidden layers - 50 nodes
#possible activation functions for hidden layers in Keras: elu (Exponential linear unit), selu (Scaled Exponential Linear Unit), 
#tanh, sigmoid, exponential, linear
#https://keras.io/activations/
model.add(layers.Dense(50, activation='relu'))
# Add another (optional):
#model.add(layers.Dense(50, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(4, activation='softmax'))

#justification for binary: 
#"we compile the model using binary cross-entropy rather than categorical cross-entropy. This may seem counterintuitive 
# for multi-label classification; however, the goal is to treat each output label as an independent Bernoulli distribution 
# and we want to penalize each output node independently."
#quoted from: https://www.pyimagesearch.com/2018/05/07/multi-label-classification-with-keras/
#more: https://stackoverflow.com/questions/42081257/keras-binary-crossentropy-vs-categorical-crossentropy-performance/46038271
model.compile(optimizer=tf.train.AdamOptimizer(0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, batch_size=150, verbose=1, shuffle=True,
         validation_data=(X_dev, Y_dev))


Train on 11853 samples, validate on 2963 samples
Epoch 1/10
11853/11853 [==============================] - 0s 37us/sample - loss: 5.7648 - acc: 0.0129 - val_loss: 3.1760 - val_acc: 0.0019
Epoch 2/10
11853/11853 [==============================] - 0s 19us/sample - loss: 3.1645 - acc: 7.5930e-04 - val_loss: 3.1739 - val_acc: 0.0014
Epoch 3/10
11853/11853 [==============================] - 0s 19us/sample - loss: 3.1540 - acc: 4.0074e-04 - val_loss: 3.1634 - val_acc: 0.0013
Epoch 4/10
11853/11853 [==============================] - 0s 19us/sample - loss: 3.1569 - acc: 2.3201e-04 - val_loss: 3.1621 - val_acc: 9.2811e-04
Epoch 5/10
11853/11853 [==============================] - 0s 18us/sample - loss: 3.1500 - acc: 6.3275e-05 - val_loss: 3.1585 - val_acc: 0.0010
Epoch 6/10
11853/11853 [==============================] - 0s 18us/sample - loss: 3.1531 - acc: 6.3275e-05 - val_loss: 3.1585 - val_acc: 7.5937e-04
Epoch 7/10
11853/11853 [==============================] - 0s 19us/sample - loss: 3.1483 -